In [ ]:
import metrics
import pandas as pd
from sklearn.metrics import f1_score

TEST_FILE = "data/test-random.tsv"
PREDICTION_FILE = "output/predictions/random_base1.1"

# Read test file
df = pd.read_csv(TEST_FILE, sep="\t", header=None)
n_col = df.shape[1]
all_targets = df[range(4, n_col)].apply(
    lambda row: row[~row.isnull()].tolist(), axis=1)
df = df[range(4)]
df.columns = ["task_name", "task_prefix", "input", "target"]
df["all_targets"] = all_targets

# Read prediction file
with open(PREDICTION_FILE) as fin:
    predictions = fin.readlines()
predictions = [x.replace("\n", "") for x in predictions]
df["prediction"] = predictions

df = df.groupby(["task_name", "task_prefix"]).agg({
    'all_targets': lambda x: list(x),
    'prediction': lambda x: list(x)}).reset_index()

def compute_metrics(row):
    return metrics.evaluate(row.prediction, row.all_targets, metrics.METRICS[row.task_name], row.task_name)

df["score"] = df.apply(compute_metrics, axis=1)
df.groupby(["task_name"]).agg({'score': ["mean", "var"]})